In [1]:
import pandas as pd
import numpy as np
import os, glob
import datetime
import matplotlib.pyplot as plt

prgname = 'gpsrsrp_segmentwise_stat.ipynb'

In [2]:
path = "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Tito/"
all_files = glob.glob(os.path.join(path, "gps_cell*.csv"))
print(len(all_files))
nfls = len(all_files)


6


In [3]:
#pick the voluteer name
name2 = all_files[0].split("/")[-2]
name = name2.split("_")[-1]
print(name)

Tito


In [4]:
cnt=0; ii=-1
dates = []
list_pnts = []
flon = np.zeros((nfls,6500)) * np.nan
flat = np.zeros((nfls,6500)) * np.nan
frsrp = np.zeros((nfls,6500)) * np.nan
for fle in all_files:
    df = pd.read_csv(fle,low_memory=True)
    numcols = len(df.columns)
    #print("fname: {0}; no. of cloumns: {1}".format(fle, numcols))
    #display(df)
    #display(df.columns)
    rte = "ROUTE_A"                             #change ROUTE A/B
    chk = df['route'].iloc[0]
    if(chk == rte):
        cnt += 1
        print("fname: {0}; no. of rows: {1}".format(fle, len(df)))
        list_pnts.append(len(df))
        date1 = fle.split("/")[-1]
        seldate = date1[28:38]
        dates.append(seldate)
        val = cnt*0.01
        df["Lon"] = df['Lon'] + val
        df.drop(["index","GPS","cputime","speed","altitude","date","time","index.1",
                "route","cellinfo","cputime.1","ncells","type","rsrq",
                "sinr","rssi","level","dcpu"], axis=1, inplace=True)
        #df = df[df['rsrp'] < -1000.]
        #df = df[(df['rsrp'] >= -156.) & (df['rsrp'] <= -31.)]
        
        print(df['Lat'].min(), df['Lon'].max())
        #lon, lat store into an array
        lonn = df['Lon'].astype(float)
        latt = df['Lat'].astype(float)
        values = df['rsrp'].astype(float)
        npts = len(lonn)
        ii += 1
        for jj in range(npts):
            #print(lonn[jj],latt[jj])
            flon[ii,jj] = lonn[jj]
            flat[ii,jj] = latt[jj]
            frsrp[ii,jj] = values[jj]

nfls = cnt
print(f"no. of files: {cnt}")
print(list_pnts)

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Tito/gps_cell_merge-null-unknown-2023-06-20-19-54-16-696000000-titoday2return.csv; no. of rows: 1679
32.89203436 -117.18649230999999
fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Tito/gps_cell_merge-null-unknown-2023-06-21-20-20-54-801000000-titoday3hometo.csv; no. of rows: 758
32.8920072 -117.17654970000001
no. of files: 2
[1679, 758]


In [5]:
outpath= "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/test/signal_prediction_plots/"
csvfname = outpath+name +'_'+rte+ '_RSRP_segmentwise_stat.csv'
print(csvfname)

/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/test/signal_prediction_plots/Tito_ROUTE_B_RSRP_segmentwise_stat.csv


In [6]:
newDataFrame = pd.DataFrame()
for k in range(1,len(list_pnts)):
    mdata = frsrp[0,:]
    data = frsrp[k,:]
    #selection of nn value
    mpnts = list_pnts[0]
    tpnts = list_pnts[1]
    if(tpnts > mpnts):
        spnts = mpnts
    else: spnts = tpnts
    nn = spnts // 30
    ndates = []; segrnge = [] 
    prmavg = []; secavg = []
    prmstd = []; secstd = []
    corrs = []
    for j in range(nn):
        stp = j*30
        etp = stp + 30
        mdat = mdata[stp:etp]
        dat = data[stp:etp]
        #print(*mdat, *dat)
        rnge = str(stp) + '-' + str(etp)
        #picking the non NaN indices
        tdd = np.where(~np.isnan(mdat))
        mdat1 = mdat[tdd]
        dat1 = dat[tdd]
        #statisctis
        mavg = np.average(mdat1); mstd = np.nanstd(mdat1)
        pavg = np.average(dat1); pstd = np.nanstd(dat1)
        corr = np.corrcoef(mdat1,dat1)
        mpcorr = corr[0,1]
        if(np.isnan(mpcorr) | ((mavg > 1000.) | (pavg > 1000.))):
            mavg = np.nan; mstd = np.nan; mpcorr=np.nan
            pavg = np.nan; pstd = np.nan
        ndates.append(dates[k])
        segrnge.append(rnge); 
        prmavg.append(mavg); secavg.append(pavg) 
        prmstd.append(mstd); secstd.append(pstd)
        corrs.append(mpcorr)
        
        #print("%s %s %8.3f %9.3f %8.3f %8.3f %6.2f" %(dates[k],rnge,mavg,pavg,mstd,pstd,mpcorr))
    csvdf = pd.DataFrame(list(zip(ndates,segrnge,prmavg,secavg,prmstd,secstd,corrs)),
                         columns=['Date','Range(secs)','prmavg','secavg','prmstd','secstd','correlation'])
    newDataFrame = newDataFrame.append(csvdf)

newDataFrame.to_csv(csvfname, index = False)    